In [ ]:
import time
import pyodbc
import pandas as pd
from alpha_vantage.timeseries import TimeSeries

api_key = "S2RDNJMGA6ZXKSXP"
symbol = "AAPL"  
interval = "1min"  
output_size = "full"  

server = "ALEXSANDER"
database = "master"
table_name = "stock_data"

conn = pyodbc.connect(
    f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
cursor = conn.cursor()

print(f"Baixando dados para {symbol}...")

ts = TimeSeries(key=api_key, output_format="pandas")
data, meta_data = ts.get_intraday(symbol=symbol, interval=interval, outputsize=output_size)

data.columns = ["Open", "High", "Low", "Close", "Volume"]

data.reset_index(inplace=True)
data.rename(columns={"date": "Date"}, inplace=True)

data["Date"] = pd.to_datetime(data["Date"])

cursor.execute(f"""
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
    BEGIN
        CREATE TABLE {table_name} (
            id INT IDENTITY(1,1) PRIMARY KEY,
            Date DATETIME,
            [Open] FLOAT,
            High FLOAT,
            Low FLOAT,
            [Close] FLOAT,
            Volume BIGINT
        )
    END
""")
conn.commit()
print(f"Tabela {table_name} verificada/criada com sucesso.")

for _, row in data.iterrows():
    cursor.execute(f"""
        INSERT INTO {table_name} (Date, [Open], High, Low, [Close], Volume)
        VALUES (?, ?, ?, ?, ?, ?)
    """, row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Volume"])

conn.commit()
print(f"Dados do ativo {symbol} inseridos com sucesso no SQL Server!")

cursor.close()
conn.close()
print("Ingestão finalizada com sucesso! 🚀")


Baixando dados para AAPL...
Tabela stock_data verificada/criada com sucesso.
Dados do ativo AAPL inseridos com sucesso no SQL Server!
Ingestão finalizada com sucesso! 🚀
